In [15]:
import torch

torch.manual_seed(2024)
B, T, C = 4, 8, 2  # Batch size, sequence length (or time), embedding size (or channels)
x = torch.randn(B, T, C)
x

tensor([[[-1.2262, -0.0093],
         [ 1.5420, -0.4657],
         [ 0.2795, -0.2610],
         [ 0.6230, -1.1561],
         [ 0.1171, -1.8865],
         [ 2.1822, -0.1930],
         [ 0.5358, -0.8898],
         [-0.3099,  0.7741]],

        [[ 0.1236, -2.1807],
         [ 0.3700,  0.4144],
         [ 1.8567,  1.9776],
         [-0.4322,  1.3667],
         [ 0.8432, -0.0421],
         [ 1.6579, -1.3085],
         [ 0.9962,  0.9391],
         [ 1.4148,  0.6343]],

        [[ 2.7266, -1.4753],
         [-1.4808,  0.0498],
         [ 1.2883, -0.6491],
         [-0.8969,  1.2634],
         [ 0.8273,  0.4594],
         [ 0.3922, -1.0767],
         [-0.0576, -0.0596],
         [ 0.2764, -0.2403]],

        [[ 0.7203, -1.4108],
         [-0.4384,  0.3551],
         [ 0.3730, -1.3050],
         [-0.7983,  1.0442],
         [-0.1227,  0.4022],
         [-1.4295, -0.5656],
         [ 0.6971,  0.1258],
         [-0.0434,  0.5366]]])

In [45]:
# version 1
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)  # (2)

x[0]

tensor([[-1.2262, -0.0093],
        [ 1.5420, -0.4657],
        [ 0.2795, -0.2610],
        [ 0.6230, -1.1561],
        [ 0.1171, -1.8865],
        [ 2.1822, -0.1930],
        [ 0.5358, -0.8898],
        [-0.3099,  0.7741]])

In [26]:
xbow[0]

tensor([[-1.2262, -0.0093],
        [ 0.1579, -0.2375],
        [ 0.1984, -0.2453],
        [ 0.3045, -0.4730],
        [ 0.2671, -0.7557],
        [ 0.5862, -0.6619],
        [ 0.5790, -0.6945],
        [ 0.4679, -0.5109]])

In [30]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a=")
print(a)
print("----")
print("b=")
print(b)
print("----")
print("c=")
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [31]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a=")
print(a)
print("----")
print("b=")
print(b)
print("----")
print("c=")
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [37]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a=")
print(a)
print("----")
print("b=")
print(b)
print("----")
print("c=")
print(c)


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [41]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (T, T) @ (B, T, C) -> (B, T, C)

In [43]:
xbow2[0]

tensor([[-1.2262, -0.0093],
        [ 0.1579, -0.2375],
        [ 0.1984, -0.2453],
        [ 0.3045, -0.4730],
        [ 0.2671, -0.7557],
        [ 0.5862, -0.6619],
        [ 0.5790, -0.6945],
        [ 0.4679, -0.5109]])

In [44]:
torch.allclose(xbow, xbow2)

True

In [48]:
# version 3
import torch.nn.functional as F
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

NameError: name 'F' is not defined